# 📘 Lesson 1 — LangChain Fundamentals for Agentic Development

In [23]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI

load_dotenv()

True

Before we can build an autonomous agent that can **reason** and **act**, we must first understand the basic tools used to build it.

LangChain does **not** work because of one magical `"agent"` function.
Its real power comes from a simple but strong idea:

> **Composability**

This lesson is fully dedicated to this core principle.


## 🔹 Core Philosophy: Composability and LCEL

LangChain is built around small components that can be **combined together**.
Each component does **one job**, and we connect them step by step.

This idea is inspired by pipelines:

* input goes in,
* transformations happen,
* output comes out.

LangChain formalizes this idea with the **LangChain Expression Language (LCEL)**.


### 🧱 The Three Core Components

In most LangChain applications, you will work with these three elements:

* **Prompt**
  Defines *what* we ask the model.

* **Model**
  The language model that generates the response.

* **Output Parser**
  Transforms the raw model output into a usable format.

A very simple chain looks like this:

```
Prompt | Model | Output Parser
```

This is the **foundation of agentic systems**.
Even complex agents are built by composing simple chains like this.

### 🔹 Building Our First Chain with LCEL

Let’s now build our **first LangChain pipeline** to see composability in action.

The goal is simple:

> Generate a short and funny tagline for a company.

> ✍️ Step 1 — Define the Prompt Template

In [4]:
promp_template = ChatPromptTemplate.from_template(
    "Generate a short, funny tagline for a company that makes: {topic}"
)

This prompt contains a **variable** called `{topic}`.
It allows us to reuse the same prompt with different inputs.


> 🧠 Step 2 — Instantiate the Model

In [6]:
model = ChatOpenAI(model="gpt-4o", temperature=0)


* We use `gpt-4o`
* `temperature=0` means:

  * more stable
  * more deterministic
  * good for reproducible behavior

> 🔄 Step 3 — Define the Output Parser

In [7]:
output_parser = StrOutputParser()

The model returns a complex object.
The `StrOutputParser` extracts **only the text**, as a simple string.

> 🔗 Step 4 — Compose the Chain with LCEL

Here, we define a processing pipeline.

In [8]:
chain = promp_template | model | output_parser

- The prompt formats the input data.

- The language model generates a response from that prompt.

- The output parser transforms the model response into a simple string.

Each component performs a single transformation.
LCEL makes these transformations explicit and ordered.

This way of composing logic is central to LangChain:

- behavior is built by composition, not by inheritance,

- complex systems emerge from simple, connected blocks.

> ▶️ Step 5 — Invoke the Chain

Here:

* we pass the input as a dictionary
* the chain runs from start to end
* we receive a clean string as output

In [9]:
topic_input = {"topic": "Agentic AI"}
result = chain.invoke(topic_input)
print(f"Input Topic: {topic_input['topic']}")
print(f"Generated Tagline: {result}")

Input Topic: Agentic AI
Generated Tagline: "Agentic AI: Because even your to-do list deserves a personal assistant with a sense of humor!"


At this stage, we did **not** build an agent yet.

But we learned something more important:

* LangChain is **modular**
* Every agent is built from **simple chains**
* LCEL makes data flow **explicit and readable**

This pattern:

```
Prompt | Model | Output Parser
```

will appear again and again when building agentic systems.

## Practical LCEL — Piping Components Together for Inputs and Outputs

In the previous section, the chain was simple:  
one input variable → one prompt → one output.

Real-world applications are rarely that simple.

Very often, a chain must work with **several pieces of information at the same time**.
For example:
- a name and a description,
- a question and some context,
- user input and retrieved data.

LCEL supports this kind of data flow through objects called **Runnables**.
Runnables allow us to **control how information moves and transforms** inside a chain.

In this section, we focus on two common Runnable patterns:
- `RunnablePassthrough`
- `RunnableParallel`

We start with the most basic one.

### Handling Multiple Inputs with RunnablePassthrough

The goal is simple:

> Take one input dictionary and make its values available to the prompt in a clean and explicit way.

We will build a small preparation step that passes data forward without modifying its meaning.

#### Step 1 — Define the Core Components

The prompt below expects **two input variables**:
- `{name}`
- `{context}`

Any chain using this prompt must provide both keys.

In [17]:
model = ChatOpenAI(model="gpt-4o", temperature=0.7)

prompt_template = ChatPromptTemplate.from_template(
    "Write a short biography about {name}. "
    "Focus on the following contribution: {context}. "
    "Keep it light and readable."
)

output_parser = StrOutputParser()

The prompt expects a dictionary like:

```python
{"name": "...", "context": "..."}
```
In practice, input data can:

- come from another chain,

- contain extra fields,

- or be produced dynamically.

So we often need an explicit preparation step to control what the prompt receives.

#### Step 2 — Passing Data Forward with RunnablePassthrough Logic

The idea behind `RunnablePassthrough` is simple: take the input from the previous step and make it available downstream.

Here, we build a small mapping that:
- receives the full input dictionary,
- extracts the fields we need,
- produces a clean dictionary for the prompt.

In [13]:
setup_and_retrieval = {
    "name": lambda x: x["name"],
    "context": lambda x: x["context"]
}

This step does not generate new information.
It only **selects and forwards** existing values.

This pattern appears very often in agentic systems:
- passing user input,
- passing tool outputs,
- passing retrieved context.

#### Step 3 — Compose the Full Chain

The data flow is now explicit:

- the input dictionary enters the setup step,
- the setup step prepares the prompt variables,
- the prompt formats a message,
- the model generates text,
- the parser extracts a clean string.

In [18]:
chain = setup_and_retrieval | promp_template | model | output_parser

In [19]:
input_data = {
    "name": "Albert Einstein",
    "context": "the theory of relativity"
}

result = chain.invoke(input_data)

print("----- Biography Generation -----")
print(f"Input: {input_data}")
print("\nGenerated Biography:")
print(result)


----- Biography Generation -----
Input: {'name': 'Albert Einstein', 'context': 'the theory of relativity'}

Generated Biography:
In the town of Ulm in Germany's land,  
A genius was born, with a mind so grand.  
Albert Einstein was his name,  
And science was his claim to fame.  

With wild hair and eyes so bright,  
He pondered the stars, both day and night.  
In physics, he found his true delight,  
Unraveling mysteries, bringing truth to light.  

E equals mc squared, he declared,  
A theory of relativity that none compared.  
Space and time, a cosmic dance,  
He changed our view with a daring glance.  

Though his mind was vast, his heart was kind,  
For peace and justice, he was inclined.  
In a world of chaos, he sought to find,  
A path of wisdom for all of humankind.  

From Germany to Princeton, his journey went,  
A life of brilliance, fully spent.  
Albert Einstein, a name we revere,  
A legacy of knowledge, forever clear.  


> **What to Take Away ?**

- The chain does not depend on where the input comes from.
- Each step has a clear responsibility.
- Data preparation is explicit and readable.

This separation becomes essential when building agents:
reasoning, tools, and memory all rely on the same idea of controlled data flow.

## Parallel Processing with RunnableParallel

Until now, all chains followed a linear path:
each component processed the output of the previous one.

Agentic systems often need a different structure.
They must look at the **same input from several angles at once**.

Typical examples include:
- producing a summary and an analysis in parallel,
- extracting facts while keeping the original context,
- generating multiple candidates before making a decision.

LCEL supports this pattern through **RunnableParallel**.
It allows one input to feed several branches at the same time.


To illustrate this idea, we will reuse a single language model,
but ask it to perform **different tasks in parallel**.

In [21]:
model = ChatOpenAI(model="gpt-4o")
output_parser = StrOutputParser()

Each branch of the pipeline is defined by its own prompt.
All of them will receive the **same input dictionary**,
but they will produce different kinds of outputs.


In [22]:
summary_prompt = ChatPromptTemplate.from_template(
    "Write a short summary about {topic}."
)

key_points_prompt = ChatPromptTemplate.from_template(
    "List three key points about {topic}."
)


These branches are then combined into a single parallel structure.
`RunnableParallel` takes care of sending the input to all branches
and collecting their outputs in a structured way.


In [24]:
parallel_chain = RunnableParallel(
    summary=summary_prompt | model | output_parser,
    key_points=key_points_prompt | model | output_parser,
)

When the chain is invoked, both branches are executed independently.
The result is a dictionary that groups all outputs under clear keys.

In [25]:
input_data = {"topic": "Agentic AI"}
result = parallel_chain.invoke(input_data)

print("----- Parallel Processing Result -----")
print("\nSummary:")
print(result["summary"])

print("\nKey Points:")
print(result["key_points"])

----- Parallel Processing Result -----

Summary:
Agentic AI refers to artificial intelligence systems that exhibit characteristics of agency, meaning they can autonomously perceive environments, make decisions, and take actions to achieve specific goals. Often designed to simulate decision-making processes similar to those of humans or animals, agentic AI systems have applications across various domains, such as robotics, autonomous vehicles, virtual personal assistants, and complex simulations. The development of agentic AI involves interdisciplinary approaches, integrating advancements in machine learning, cognitive science, and robotics. With the potential to perform tasks without direct human intervention, agentic AI raises important ethical and safety considerations, including concerns about accountability, transparency, and the impact on employment and society.

Key Points:
Agentic AI refers to artificial intelligence systems endowed with some level of agency, meaning they can ma

The data flow is now easy to reason about:

- one input enters the system,
- several transformations happen in parallel,
- results are gathered without conflict.

This structured parallelism is essential for agents.
Before acting, an agent may need to:
- analyze a situation,
- check constraints,
- generate possible actions.

`RunnableParallel` makes this pattern explicit and manageable.